# A Python Version of GEE code to create Hotter Drought data

In [97]:
import os

import ee
import geemap.foliumap as geemap
import geemap

from find_set_root import find_set_project_root
PROJECT_ROOT = find_set_project_root()
print(f"Project root found at: {PROJECT_ROOT}")
import utils.general_functions as ugf


DIR_RAW = os.path.join(PROJECT_ROOT, 'data', 'raw')
DIR_DERIVED = os.path.join(PROJECT_ROOT, 'data', 'derived')
ugf.dir_ensure([DIR_RAW, DIR_DERIVED])

#Prepare to use Earth Engine
ee.Authenticate()
ee.Initialize(project = 'ee-tymc5571-multi-disturbance')


#####################################
# USER PARAMETERS

foldername = 'GEE_Exports'
asset_folder = 'projects/ee-tymc5571-multi-disturbance/assets/'
#####################################

Project root found at: C:\Users\tymc5571\dev\forest-disturbance-stack-v3
✅ Directory already exists: C:\Users\tymc5571\dev\forest-disturbance-stack-v3\data\raw
✅ Directory already exists: C:\Users\tymc5571\dev\forest-disturbance-stack-v3\data\derived


In [98]:
# DATA

# Load TerraClimate image collection
terraclimate = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")

# Define geometries
conus = ee.Geometry.Polygon([
    [[-130.44414062500002, 49.685186433002514],
     [-130.44414062500002, 23.508285084424926],
     [-62.15312500000001, 23.508285084424926],
     [-62.15312500000001, 49.685186433002514]]
])

colorado = ee.Geometry.Polygon([
    [[-109.06103589975199, 41.026450911300536],
     [-109.06103589975199, 37.27942746087441],
     [-101.94189527475199, 37.27942746087441],
     [-101.94189527475199, 41.026450911300536]]
])

west = ee.Geometry.Polygon([
    [[-124.96923902475199, 48.90527784208563],
     [-124.96923902475199, 31.855302630433012],
     [-113.80712964975199, 31.855302630433012],
     [-113.80712964975199, 48.90527784208563]]
])

# Import NEON domains
neondomainsconus = ee.FeatureCollection("users/tymc5571/NEON_Domains_CONUS")



# Filter domain
domain = neondomainsconus.filter(
    ee.Filter.Or(
        ee.Filter.eq("DomainName", 'Northern Rockies'),
        ee.Filter.eq("DomainName", 'Great Basin'),
        ee.Filter.eq("DomainName", 'Pacific Northwest'),
        ee.Filter.eq("DomainName", 'Pacific Southwest'),
        ee.Filter.eq("DomainName", 'Desert Southwest'),
        ee.Filter.eq("DomainName", 'Southern Rockies / Colorado Plateau')
    )
)

# # Debug print statements
# print('Domain:', domain.getInfo())

# # View terraclimate structure
# print('All TerraClimate:', terraclimate.limit(1).getInfo())  # limit to avoid printing the whole collection

# # Example visualization image
# tmmx_example = terraclimate.select('tmmx') \
#     .filter(ee.Filter.calendarRange(1, 1, 'month')) \
#     .filter(ee.Filter.calendarRange(1958, 1958, 'year')) \
#     .mean()  # Just for demonstration

# VARIABLES

# List of variable names
VARIABLE_NAMES = ee.List(['tmmx', 'vpd', 'def', 'soil', 'pr', 'pdsi'])

# Warm/dry direction dictionary
directions = ['positive', 'positive', 'positive', 'negative', 'negative', 'negative']
direction_dictionary = ee.Dictionary.fromLists(VARIABLE_NAMES, directions)
print("Direction dictionary:", direction_dictionary.getInfo())

# Thresholds
warmDryTAvg = ee.List([0.37, 0.30, 0.49, -0.21, -0.39, -0.73])
warmDryTMax = ee.List([0.41, 0.34, 0.53, -0.24, -0.42, -0.77])
warmDryTMin = ee.List([0.33, 0.26, 0.45, -0.18, -0.36, -0.69])

# Use opposite thresholds for cold/wet
def flip(x):
    """Function to flip the sign of a number."""
    return ee.Number(x).multiply(ee.Number(-1))

coldWetTAvg = warmDryTAvg.map(flip);
coldWetTMax = warmDryTMax.map(flip);
coldWetTMin = warmDryTMin.map(flip);


# Minimum number of variables to trigger binary outcome
numVarThreshold = ee.Number(0)

# Years of interest
years = ee.List.sequence(1958, 2023)

# Output
print('Years:', years)


Direction dictionary: {'def': 'positive', 'pdsi': 'negative', 'pr': 'negative', 'soil': 'negative', 'tmmx': 'positive', 'vpd': 'positive'}
Years: ee.List({
  "functionInvocationValue": {
    "functionName": "List.sequence",
    "arguments": {
      "end": {
        "constantValue": 2023
      },
      "start": {
        "constantValue": 1958
      }
    }
  }
})


In [99]:
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////
#////////////////////////////////// DROUGHT ANALYSIS - DO NOT CHANGE BELOW /////////////////////////////////////
#///////////////////////////////////////////////////////////////////////////////////////////////////////////////

#///////////////////////////// SET UP FOR ANALYSIS ////////////////////

# List of months (1 to 12)
months = ee.List.sequence(1, 12)

# Select only variables of interest
vars = terraclimate.select(VARIABLE_NAMES)

# Function to add a 'month' band to each image
def add_month_band(image):
    month_val = image.date().get('month')
    month_band = ee.Image.constant(month_val).toInt().rename('month')
    return image.addBands(month_band)

# Map over the image collection to add month band
vars_with_month = vars.map(add_month_band)

In [100]:
# ///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
# ////////////// CALCULATE MONTHLY AVERAGES OVER CLIMATE HISTORY, THEN GET MONTHS OF "TYPICALLY DRIEST/WARMEST AND WETTEST/COLDEST"
# ///////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# /////// FUNCTIONS //////

# Function to pull monthly data over entire time frame, average, and reset image properties/band names
def get_month_mean(month):
    month = ee.Number(month)
    means = vars_with_month.filter(ee.Filter.calendarRange(month, month, 'month')).mean()

    # Create a list of names with _mean appended
    meannamelist = ee.List(VARIABLE_NAMES).map(lambda name: ee.String(name).cat('_mean'))
    meannamelist = meannamelist.add('mean_month')

    means = means.rename(meannamelist)
    means = means.set('month', month.toInt())
    means = means.set('monthclean', month.toInt().format('%02d'))
    return means

# Function to create a min reducer that retains band names & values for all other bands, append 'min' to band names
def argmin_reduce(image_collection):
    band_names = image_collection.first().bandNames()
    reducer = ee.Reducer.min(band_names.size()).setOutputs(band_names)
    output = image_collection.reduce(reducer)

    # Rename bands with _min suffix
    new_names = output.bandNames().map(lambda name: ee.String(name).cat('_min'))
    output = output.rename(new_names)
    return output

# Function to create a max reducer that retains band names & values for all other bands, append 'max' to band names
def argmax_reduce(image_collection):
    band_names = image_collection.first().bandNames()
    reducer = ee.Reducer.max(band_names.size()).setOutputs(band_names)
    output = image_collection.reduce(reducer)

    # Rename bands with _max suffix
    new_names = output.bandNames().map(lambda name: ee.String(name).cat('_max'))
    output = output.rename(new_names)
    return output

# Function to calculate both minimum and maximum means and the months of occurrence
def minmax_mean(variable):
    variable = ee.String(variable)
    var_mean_name = variable.cat('_mean')
    var_mean_month_name = variable.cat('_mean_month')

    var_means = monthmeans.select([var_mean_name, 'mean_month'])

    # Rename mean_month to include variable name
    var_means = var_means.map(lambda img: img.rename([var_mean_name, var_mean_month_name]))

    min_means = argmin_reduce(var_means)
    max_means = argmax_reduce(var_means)

    minmax = min_means.addBands(max_means)
    minmax = minmax.set('variable', variable)
    return minmax

# //////// PERFORM TYPICALLY WARMEST/DRIEST & COLDEST/WETTEST ANALYSIS ////////

# Create list of images for each month (monthly means)
monthmeans = ee.ImageCollection.fromImages(months.map(get_month_mean))

# Map minmax_mean over all variable names
allminmax = ee.ImageCollection.fromImages(ee.List(VARIABLE_NAMES).map(minmax_mean))

# Turn the collection into a single multi-band image
allminmax = allminmax.toBands()

# Clean up band names: remove "1_" prefixes
def clean_bandname(name):
    return ee.String(name).slice(2)

allminmax = allminmax.rename(allminmax.bandNames().map(clean_bandname))

# At this point, you can export or visualize `allminmax`
# For example, to export or analyze: allminmax.select('tmmx_mean_min'), etc.

# Optional debug print (truncated for readability)
print("Values and months of warmest/driest and coldest/wettest (bands):", allminmax.bandNames().getInfo())


# #Test layers for map
# Map4 = geemap.Map(center=[50, -130], zoom=5)


# Map.addLayer(allminmax.select('tmmx_mean_min'), {'min': -100, 'max': 500, 'palette':['blue', 'red']}, 'temp_min_mean');
# Map.addLayer(allminmax.select('tmmx_mean_month_min'), {'min': 1, 'max': 12, 'palette':['blue', 'red']}, 'temp_min_mean_month');
# Map.addLayer(allminmax.select('tmmx_mean_max'), {'min': -100, 'max': 500, 'palette':['blue', 'green']}, 'temp_max_mean');
# Map.addLayer(allminmax.select('tmmx_mean_month_max'), {'min': 1, 'max': 12, 'palette':['blue', 'red']}, 'temp_max_mean_month');
# Map


Values and months of warmest/driest and coldest/wettest (bands): ['tmmx_mean_min', 'tmmx_mean_month_min', 'tmmx_mean_max', 'tmmx_mean_month_max', 'vpd_mean_min', 'vpd_mean_month_min', 'vpd_mean_max', 'vpd_mean_month_max', 'def_mean_min', 'def_mean_month_min', 'def_mean_max', 'def_mean_month_max', 'soil_mean_min', 'soil_mean_month_min', 'soil_mean_max', 'soil_mean_month_max', 'pr_mean_min', 'pr_mean_month_min', 'pr_mean_max', 'pr_mean_month_max', 'pdsi_mean_min', 'pdsi_mean_month_min', 'pdsi_mean_max', 'pdsi_mean_month_max']


In [101]:
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
# ////////////// CALCULATE CLIMATE ANOMALIES FOR THE TYPICALLY DRIEST/WARMEST MONTH IN EACH YEAR /////////////////////
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////
# // FUNCTION TO GET ANOMALIES RESULTING IN AN IMAGE COLLECTION WITH AN IMAGE FOR EACH VARIABLE, YEARS ARE BANDS //
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////

def get_all_anomalies_for_variable(variable):
    variable = ee.String(variable)
    varlen = variable.length()
    dir = ee.String(direction_dictionary.get(variable))
    
    # Get band name for min or max depending on direction
    varname = ee.Algorithms.If(
        dir.equals('positive'),
        variable.cat('_mean_max'),
        variable.cat('_mean_min')
    )
    varname = ee.String(varname)

    # Get month band name for min/max
    monthname = ee.Algorithms.If(
        dir.equals('positive'),
        variable.cat('_mean_month_max'),
        variable.cat('_mean_month_min')
    )
    monthname = ee.String(monthname)

    # Function to get anomaly for one year and one variable
    def get_anomaly_one_year(year):
        year = ee.Number(year)

        # Function to get anomaly for one month
        def get_anomaly_one_month(month):
            month = ee.Number(month)

            dats = ee.Image(
                vars_with_month
                .filter(ee.Filter.calendarRange(year, year, 'year'))
                .filter(ee.Filter.calendarRange(month, month, 'month'))
                .select(variable)
                .toList(1)
                .get(0)
            )

            anom = allminmax.select(monthname)
            anom = anom.remap([month], [1])  # map current month to 1 (true), else masked
            anom = anom.multiply(dats.select(variable).subtract(allminmax.select(varname)))

            return anom

        # Map anomaly calculation for each month in year
        month_anoms = months.map(get_anomaly_one_month)

        # Blend images for all 12 months into one image
        blended = ee.Image(month_anoms.get(0))
        for i in range(1, 12):
            blended = blended.blend(ee.Image(month_anoms.get(i)))

        # Rename blended image band to the variable name (note: keeping single band per year)
        yrstring = year.format('%04d')
        blended = blended.rename(variable)
        blended = blended.set('year', yrstring)

        return blended

    # Map over all years
    all_year_anoms = years.map(get_anomaly_one_year)
    return all_year_anoms

# ///////// Map function to get anomalies over all variables included in the variable_names list /////////

allanomsraw = VARIABLE_NAMES.map(get_all_anomalies_for_variable)
print("Raw anomalies: ", allanomsraw.getInfo())

# Example visualization (cast correctly from nested list)
viz = ee.Image(ee.List(allanomsraw.get(0)).get(0))  # First variable, first year
print('One raster of raw anomalies (one year, one variable):', viz.getInfo())


# Map2 = geemap.Map(center=[50, -130], zoom=5)
# Map2.addLayer(viz, {'min': -50, 'max': 50, 'palette': ['blue', 'red']}, 'anom viz test yr 1')
# Map2


Raw anomalies:  [[{'type': 'Image', 'bands': [{'id': 'tmmx', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -65535, 'max': 65535}, 'crs': 'GEOGCS["unknown", \n  DATUM["unknown", \n    SPHEROID["Spheroid", 6378137.0, 298.257223563]], \n  PRIMEM["Greenwich", 0.0], \n  UNIT["degree", 0.017453292519943295], \n  AXIS["Longitude", EAST], \n  AXIS["Latitude", NORTH]]', 'crs_transform': [0.041666666666666664, 0, -180, 0, -0.041666666666666664, 90]}], 'properties': {'year': '1958'}}, {'type': 'Image', 'bands': [{'id': 'tmmx', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -65535, 'max': 65535}, 'crs': 'GEOGCS["unknown", \n  DATUM["unknown", \n    SPHEROID["Spheroid", 6378137.0, 298.257223563]], \n  PRIMEM["Greenwich", 0.0], \n  UNIT["degree", 0.017453292519943295], \n  AXIS["Longitude", EAST], \n  AXIS["Latitude", NORTH]]', 'crs_transform': [0.041666666666666664, 0, -180, 0, -0.041666666666666664, 90]}], 'properties': {'year': '1959'}}, {'type': 'Image', 'ban

In [102]:
# /////////////////////////////////////////////////////////////////////////////
# ////////////////// STANDARDIZE ANOMALIES INTO Z-SCORES //////////////////////
# /////////////////////////////////////////////////////////////////////////////

# A function to standardize a list of images into Z-scores; returns a list of images
# Takes in any image list and standardizes all images pixel-wise.
# E.g. for 10 images of data (e.g., years), the data for a single location will be standardized

def standardize_list(img_list):
    img_list = ee.List(img_list)
    image_collection = ee.ImageCollection.fromImages(img_list)  # Convert to ImageCollection

    # Calculate pixel-wise standard deviation and mean across the stack
    stack_sd = image_collection.reduce(ee.Reducer.stdDev())
    stack_mean = image_collection.reduce(ee.Reducer.mean())

    # Function to calculate z-score for one image
    def calculate_z_score(image):
        image = ee.Image(image)
        z_scored = image.subtract(stack_mean).divide(stack_sd)
        yrprop = image.get('year')
        return z_scored.set('year', yrprop)

    # Map over the collection server-side
    standardized = image_collection.map(calculate_z_score)

    # Return as list to match input style (list in, list out)
    standardized_list = standardized.toList(img_list.length())
    return standardized_list

# ///////////////////////////////////////////////////////////////////
# // Map standardization function over all anomaly variable sets ///
# ///////////////////////////////////////////////////////////////////

# allanomsraw is a list of lists: one list per variable, each containing images (years)
standardizeAll = ee.List(allanomsraw).map(standardize_list)  # Server-side map over variable list

std_img = ee.Image(ee.List(standardizeAll.get(0)).get(62))  # 0 = first variable, 62 = 2020 if years are 1958-2021

Map = geemap.Map(center=[39, -105], zoom=5)  # Center over Colorado, adjust as needed
Map.addLayer(std_img, {'min': -3, 'max': 3, 'palette': ['blue', 'white', 'red']}, 'Standardized Anomaly 2020')
Map

# testSD = ee.ImageCollection.fromImages(allanomsraw.get(0)).reduce(ee.Reducer.stdDev());
# Map3 = geemap.Map(center=[0, 0], zoom=2)
# Map3.addLayer(testSD, {'min': 5, 'max': 20, 'palette':['blue', 'red']}, 'tmmxSD');
# Map3



# Create an imageCollection out of the z-scores
year_band_names = years.map(lambda y: ee.String('yr_').cat(ee.Number(y).format('%04d')))

def stack_years_to_bands(img_list, var_idx):
    # img_list: list of images for one variable (length = number of years)
    img_list = ee.List(img_list)
    # Stack all years as bands
    stacked = ee.ImageCollection.fromImages(img_list).toBands()

    # Set variable name as property
    var_name = ee.String(VARIABLE_NAMES.get(var_idx))

     # Build new band names with prefix
    new_band_names = year_band_names.map(
        lambda name: ee.String('zscore_').cat(var_name).cat('_').cat(name)
    )

    # Rename bands
    stacked = stacked.rename(new_band_names)

    return stacked.set('variable', var_name)

# Map over variables (standardizeAll is a list of lists: [var][year])
stacked_per_var = ee.List.sequence(0, VARIABLE_NAMES.length().subtract(1)).map(
    lambda idx: stack_years_to_bands(standardizeAll.get(idx), idx)
)

# Convert to ImageCollection: each image is a variable, bands are years
standardizeAll_IC = ee.ImageCollection.fromImages(stacked_per_var)

print(standardizeAll_IC.size().getInfo(), "images in standardized collection")

print(standardizeAll_IC.first().bandNames().getInfo())  # Print band names of the first image

# # Example: visualize the first variable (all years as bands)
# tmmxImg = ee.Image(standardizeAll_IC.first())
# print('Bands:', img.bandNames().getInfo())

# test = tmmxImg.select('zscore_tmmx_yr_2020')  # Select the first variable (tmmx) for visualization
# Map3 = geemap.Map(center=[50, -130], zoom=5)
# Map3.addLayer(test, {'min': -2, 'max': 2, 'palette':['blue', 'red']}, 'tmmx yr1 z-scores');
# Map3


6 images in standardized collection
['zscore_tmmx_yr_1958', 'zscore_tmmx_yr_1959', 'zscore_tmmx_yr_1960', 'zscore_tmmx_yr_1961', 'zscore_tmmx_yr_1962', 'zscore_tmmx_yr_1963', 'zscore_tmmx_yr_1964', 'zscore_tmmx_yr_1965', 'zscore_tmmx_yr_1966', 'zscore_tmmx_yr_1967', 'zscore_tmmx_yr_1968', 'zscore_tmmx_yr_1969', 'zscore_tmmx_yr_1970', 'zscore_tmmx_yr_1971', 'zscore_tmmx_yr_1972', 'zscore_tmmx_yr_1973', 'zscore_tmmx_yr_1974', 'zscore_tmmx_yr_1975', 'zscore_tmmx_yr_1976', 'zscore_tmmx_yr_1977', 'zscore_tmmx_yr_1978', 'zscore_tmmx_yr_1979', 'zscore_tmmx_yr_1980', 'zscore_tmmx_yr_1981', 'zscore_tmmx_yr_1982', 'zscore_tmmx_yr_1983', 'zscore_tmmx_yr_1984', 'zscore_tmmx_yr_1985', 'zscore_tmmx_yr_1986', 'zscore_tmmx_yr_1987', 'zscore_tmmx_yr_1988', 'zscore_tmmx_yr_1989', 'zscore_tmmx_yr_1990', 'zscore_tmmx_yr_1991', 'zscore_tmmx_yr_1992', 'zscore_tmmx_yr_1993', 'zscore_tmmx_yr_1994', 'zscore_tmmx_yr_1995', 'zscore_tmmx_yr_1996', 'zscore_tmmx_yr_1997', 'zscore_tmmx_yr_1998', 'zscore_tmmx_yr_1999

In [103]:


# //////////////////////////////////////////////////////////////
# ////////////// THRESHOLD STANDARDIZED ANOMALIES //////////////
# //////////////////////////////////////////////////////////////

# Zip up thresholds
avgThresholds = warmDryTAvg.zip(coldWetTAvg)
minThresholds = warmDryTMin.zip(coldWetTMin)
maxThresholds = warmDryTMax.zip(coldWetTMax)

# Function to threshold z scores
def threshold_z_scores(dats):
    dats = ee.List(dats)
    zAnoms = ee.List(dats.get(0))
    thresholds = ee.List(dats.get(1))
    warmT = ee.Number(thresholds.get(0))
    coldT = ee.Number(thresholds.get(1))

    def threshold_image(img):
        img = ee.Image(img)
        imYr = img.get('year')

        warmMask = ee.Algorithms.If(
            warmT.lt(ee.Number(0)),
            img.lte(warmT),
            img.gte(warmT)
        )
        warmMask = ee.Image(warmMask).set('year', imYr)

        coldMask = ee.Algorithms.If(
            coldT.lt(ee.Number(0)),
            img.lte(coldT),
            img.gte(coldT)
        )
        cold = ee.Image(coldMask).remap([ee.Number(1), ee.Number(0)], [ee.Number(-1), ee.Number(0)])
        cold = cold.set('year', imYr)

        return ee.List([warmMask, cold])

    thresholdImages = zAnoms.map(threshold_image)
    return thresholdImages

# Run for each set of thresholds
avgThresholdImages = standardizeAll.zip(avgThresholds).map(threshold_z_scores)
minThresholdImages = standardizeAll.zip(minThresholds).map(threshold_z_scores)
maxThresholdImages = standardizeAll.zip(maxThresholds).map(threshold_z_scores)

# Function to zip a list containing lists with one element per year
def zip_list(masterList):
    numYears = ee.List(years).length()
    indices = ee.List.sequence(0, numYears.subtract(1))

    def zip_index(index):
        masterList_ = ee.List(masterList)
        collected = masterList_.map(lambda l: ee.List(l).get(index))
        return collected

    zippedList = indices.map(zip_index)
    return zippedList

# Zip up all of the threshold lists
zippedAvgThresholdImages = zip_list(avgThresholdImages)
zippedMinThresholdImages = zip_list(minThresholdImages)
zippedMaxThresholdImages = zip_list(maxThresholdImages)

# Function to produce fingerprint images per year
def make_fingerprint(yearData):
    yearData = ee.List(yearData)

    var1warm = ee.Image(ee.List(yearData.get(0)).get(0))
    var2warm = ee.Image(ee.List(yearData.get(1)).get(0))
    var3warm = ee.Image(ee.List(yearData.get(2)).get(0))
    var4warm = ee.Image(ee.List(yearData.get(3)).get(0))
    var5warm = ee.Image(ee.List(yearData.get(4)).get(0))
    var6warm = ee.Image(ee.List(yearData.get(5)).get(0))
    var1cold = ee.Image(ee.List(yearData.get(0)).get(1))
    var2cold = ee.Image(ee.List(yearData.get(1)).get(1))
    var3cold = ee.Image(ee.List(yearData.get(2)).get(1))
    var4cold = ee.Image(ee.List(yearData.get(3)).get(1))
    var5cold = ee.Image(ee.List(yearData.get(4)).get(1))
    var6cold = ee.Image(ee.List(yearData.get(5)).get(1))

    fYr = var1warm.get('year')

    warmF = var1warm.add(var2warm).add(var3warm).add(var4warm).add(var5warm).add(var6warm)
    warmF = warmF.rename('warmFingerprint')

    coldF = var1cold.add(var2cold).add(var3cold).add(var4cold).add(var5cold).add(var6cold)
    coldF = coldF.rename('coldFingerprint')

    fingerprint = warmF.addBands(coldF)
    fingerprint = fingerprint.set('year', fYr)
    return fingerprint

# Make fingerprints
fingerprintAvg = zippedAvgThresholdImages.map(make_fingerprint)
fingerprintMin = zippedMinThresholdImages.map(make_fingerprint)
fingerprintMax = zippedMaxThresholdImages.map(make_fingerprint)

# Function to clean fingerprints
def cleanFingerprint(yearImg):
    yearImg = ee.Image(yearImg)
    warmMask = yearImg.select("warmFingerprint").gte(numVarThreshold)
    coldMask = yearImg.select("coldFingerprint").lte(numVarThreshold.multiply(ee.Number(-1)))
    cleanWarm = yearImg.select("warmFingerprint").updateMask(warmMask)
    cleanCold = yearImg.select("coldFingerprint").updateMask(coldMask)
    return cleanWarm.addBands(cleanCold)

# Create clean fingerprints
cleanFAvg = fingerprintAvg.map(cleanFingerprint)
cleanFMax = fingerprintMax.map(cleanFingerprint)
cleanFMin = fingerprintMin.map(cleanFingerprint)

# Convert to ImageCollections
ICcleanFAvg = ee.ImageCollection.fromImages(cleanFAvg)
ICcleanFMax = ee.ImageCollection.fromImages(cleanFMax)
ICcleanFMin = ee.ImageCollection.fromImages(cleanFMin)


# Sample PDSI comparison
yr = ee.Number(2020)
summerPDSI = terraclimate.select('pdsi') \
    .filter(ee.Filter.calendarRange(7, 8, 'month')) \
    .filter(ee.Filter.calendarRange(yr, yr, 'year')) \
    .mean()
summerPDSImask4 = summerPDSI.lte(-400) # -3 is often considered severe drought, while -4 is extreme drought - https://climatedataguide.ucar.edu/climate-data/palmer-drought-severity-index-pdsi
summerPDSImask3 = summerPDSI.lte(-300) # -3 is often considered severe drought, while -4 is extreme drought - https://climatedataguide.ucar.edu/climate-data/palmer-drought-severity-index-pdsi
summerPDSI4 = summerPDSI.updateMask(summerPDSImask4)
summerPDSI3 = summerPDSI.updateMask(summerPDSImask3)


# # Print one of the fingerprints for one year & PDSI
# Map4 = geemap.Map(center=[40, -120], zoom=5)
# Map4.addLayer(ee.Image(cleanFAvg.get(62)).select("warmFingerprint"), {'min': 0, 'max': 6, 'palette':['blue', 'red']}, 'warmFingerprintClean2020')
# Map4.addLayer(ee.Image(cleanFMax.get(62)).select("warmFingerprint"), {'min': 0, 'max': 6, 'palette':['blue', 'red']}, 'warmFingerprintMaxClean2020')
# Map4.addLayer(ee.Image(summerPDSI4), {'min': -500, 'max': 0, 'palette':['orange', 'blue']}, 'PDSI summer 2020 - Extreme Drought')
# Map4.addLayer(ee.Image(summerPDSI3), {'min': -500, 'max': 0, 'palette':['orange', 'blue']}, 'PDSI summer 2020 - Severe Drought')    
# Map4.addLayer(ee.Image(summerPDSI), {'min': -500, 'max': 0, 'palette':['orange', 'blue']}, 'PDSI summer 2020 - All')
# Map4

# Convert the ImageCollection to a list
IC_list_fingerprint_avg = ICcleanFAvg.toList(cleanFAvg.size())

# Start with the first image, renamed
first = ee.Image(IC_list_fingerprint_avg.get(0)).select('warmFingerprint') \
    .rename(ee.String('hd_fingerprint_yr_').cat(ee.Number(years.get(0)).format('%d')))

# Function to stack each year's image as a new band
def stack_year_band(index, prev):
    index = ee.Number(index)
    year = years.get(index)
    img = ee.Image(IC_list_fingerprint_avg.get(index)).select('warmFingerprint')
    renamed = img.rename(ee.String('hd_fingerprint_yr_').cat(ee.Number(year).format('%d')))
    return ee.Image(prev).addBands(renamed)

# Run iteration from index 1 onward
warm_fingerprint_image = ee.Image(
    ee.List.sequence(1, years.size().subtract(1)).iterate(stack_year_band, first)
)

print(warm_fingerprint_image.bandNames().getInfo())


['hd_fingerprint_yr_1958', 'hd_fingerprint_yr_1959', 'hd_fingerprint_yr_1960', 'hd_fingerprint_yr_1961', 'hd_fingerprint_yr_1962', 'hd_fingerprint_yr_1963', 'hd_fingerprint_yr_1964', 'hd_fingerprint_yr_1965', 'hd_fingerprint_yr_1966', 'hd_fingerprint_yr_1967', 'hd_fingerprint_yr_1968', 'hd_fingerprint_yr_1969', 'hd_fingerprint_yr_1970', 'hd_fingerprint_yr_1971', 'hd_fingerprint_yr_1972', 'hd_fingerprint_yr_1973', 'hd_fingerprint_yr_1974', 'hd_fingerprint_yr_1975', 'hd_fingerprint_yr_1976', 'hd_fingerprint_yr_1977', 'hd_fingerprint_yr_1978', 'hd_fingerprint_yr_1979', 'hd_fingerprint_yr_1980', 'hd_fingerprint_yr_1981', 'hd_fingerprint_yr_1982', 'hd_fingerprint_yr_1983', 'hd_fingerprint_yr_1984', 'hd_fingerprint_yr_1985', 'hd_fingerprint_yr_1986', 'hd_fingerprint_yr_1987', 'hd_fingerprint_yr_1988', 'hd_fingerprint_yr_1989', 'hd_fingerprint_yr_1990', 'hd_fingerprint_yr_1991', 'hd_fingerprint_yr_1992', 'hd_fingerprint_yr_1993', 'hd_fingerprint_yr_1994', 'hd_fingerprint_yr_1995', 'hd_fingerp

In [104]:
#Export clean fingerprints to assets
task_hdwf = ee.batch.Export.image.toAsset(
    image=warm_fingerprint_image,
    description='hd_warm_fingerprint',
    assetId=asset_folder + 'hd_warm_fingerprint',
    scale=4638.3,  # adjust as needed
    region=domain.geometry().bounds().getInfo()['coordinates'], 
    maxPixels=1e13
)
task_hdwf.start()


In [105]:

# Export standardizeAll_IC to assets
standardizeAll_IC_list = standardizeAll_IC.toList(standardizeAll_IC.size())
for i in range(standardizeAll_IC.size().getInfo()):
    img = ee.Image(standardizeAll_IC_list.get(i))
    var = img.get('variable').getInfo()
    asset_id = f'{asset_folder}hd_zscores_{var}'
    task_hdz = ee.batch.Export.image.toAsset(
        image=img,
        description=f'hd_zscores_{var}',
        assetId=asset_id,
        scale=4638.3,  # adjust as needed
        region=domain.geometry().bounds().getInfo()['coordinates'],
        maxPixels=1e13
    )
    task_hdz.start()



In [106]:


# OLD ASSET EXPORT CODE - NOT USED

# // /////////////////////////////////////////////////////////
# // ///////////// EXPORT AS ASSET FOR USE ///////////////////
# // /////////////////////////////////////////////////////////

# // // Retain only warm fingerprint, clip to domain
# // var cleanAvgWarm = ICcleanFAvg.select('warmFingerprint').filterBounds(domain);
# // print("To Export:", cleanAvgWarm);
# // Map.addLayer(ee.Image(cleanAvgWarm.first()).clip(domain), {min: 0, max: 6, palette:['blue', 'red']}, 'testClip', false);

# // // We cannot use 'map' to export images, as Export is a client side function. Instead we have to use a loop (yuck)
# // // Here we use an adapted version of  batch download from fitoprincipe repo: https://github.com/fitoprincipe/geetools-code-editor

# // // CODE INDENTED BELOW TAKEN FROM FITOPRINCIPE REPO & ADAPTED AS NOTED IN CODE

# //             var tools = require('users/fitoprincipe/geetools:tools');
# //             var helpers = require('users/fitoprincipe/geetools:helpers_js');
            
# //             var getRegion = function(object, bounds) {
# //               bounds = bounds || false;
# //               try {
# //                 var name = object.name();
# //                 if (name in ['Image', 'Feature', 'ImageCollection', 'FeatureCollection']) {
# //                   var geom = object.geometry();
# //                 } else {
# //                   var geom = object;
# //                 }
# //                 if (bounds) {
# //                   geom = geom.bounds();
# //                 }
# //                 return geom;
# //               } catch(err) {
# //                 print(err.message);
# //                 return object;
# //               }
# //             };
            
# //             exports.getRegion = getRegion;
            
            
            
# //             // TASK CLASS
# //             var Task = function(taskId, config) {
# //               this.id = taskId;
# //               this.config = config;
# //             };
            
# //             Task.prototype.start = function() {
# //                 ee.data.startProcessing(this.id, this.config);
# //             };
            
# //             var IMAGE_TYPES = function(img, type) {
# //             var types = {  "float":img.toFloat(), 
# //                             "byte":img.toByte(), 
# //                             "int":img.toInt(),
# //                             "double":img.toDouble(),
# //                             "long": img.toLong(),
# //                             "short": img.toShort(),
# //                             "int8": img.toInt8(),
# //                             "int16": img.toInt16(),
# //                             "int32": img.toInt32(),
# //                             "int64": img.toInt64(),
# //                             "uint8": img.toUint8(),
# //                             "uint16": img.toUint16(),
# //                             "uint32": img.toUint32()};
              
# //               return types[type];
# //             };
            
# //             var Download = {'ImageCollection': {}, 'Table': {}, 'Image':{}};
            
# //             Download.ImageCollection.toAsset = function(collection, assetFolder, options) {
# //               var root = ee.data.getAssetRoots()[0]['id'];
# //               var folder = assetFolder;
# //               if (folder !== null && folder !== undefined) {
# //                 var assetFolder = root+'/'+folder+'/';
# //               } else {
# //                 var assetFolder = root+'/';
# //               }
              
# //               var defaults = {
# //                   name: null,
# //                   scale: 1000,
# //                   maxPixels: 1e13,
# //                   region: null
# //                 };
                
# //               var opt = tools.get_options(defaults, options);
# //               var n = collection.size().getInfo();
                
# //               var colList = collection.toList(n);
              
# //               for (var i = 0; i < n; i++) {
# //                 var img = ee.Image(colList.get(i));
# //                 var id = img.id().getInfo() || 'image_'+i.toString();
            
# //                 var yr = img.get('year');                                       ////////// CHANGED
# //                 var nm = ee.String('HotterDrought_').cat(yr).getInfo();        ////////// CHANGED
# //                 var region = opt.region || img.geometry().bounds().getInfo()["coordinates"];
# //                 var assetId = assetFolder+nm;                                  ////////// CHANGED
                
# //                 Export.image.toAsset({
# //                   image: img,
# //                   description: nm,                                             ////////// CHANGED
# //                   assetId: assetId,
# //                   region: region,
# //                   scale: opt.scale,
# //                   maxPixels: opt.maxPixels});
# //               }
# //             };


# // // Run batch export -- it takes a while to run since it's client-side, so be patient in waiting for tasks to appear
# // // var batch = require('users/fitoprincipe/geetools:batch');
# // var asset = 'HotterDrought';
# // var options = {
# //   name: 'HotterDrought_{year}',
# //   scale: 4638.3,
# //   region: domain
# // };
# // // batch.Download.ImageCollection.toAsset(cleanAvgWarm, asset, options); <- adapted batch script above to allow for correct naming scheme. Instead run on line below
# // Download.ImageCollection.toAsset(cleanAvgWarm, asset, options);




# // // // THIS WORKS for batch-exporting calculated monthmeans
# // // //var batch = require('users/fitoprincipe/geetools:batch');
# // // var asset = 'testMonthMeans';
# // // var options = {
# // //   name: 'MonthMeans_{monthclean}',
# // //   scale: 4638.3,
# // //   region: domain
# // // };
# // // //batch.Download.ImageCollection.toAsset(monthmeans, asset, options);
# // // Download.ImageCollection.toAsset(monthmeans, asset, options);



# OLD CODE FOR EXPORTING A VIDEO

# // ///////////////////////////////////////////////////////////////////
# // /////////////////////// VISUALIZE OUTPUT //////////////////////////
# // ///////////////////////////////////////////////////////////////////

# // // var warmViz = ee.ImageCollection(binaryFMax.select('warmFingerprint'));

# // // //Args for video
# // // var videoArgs = {
# // //   dimensions: 768,
# // // //  region: colorado,
# // //   region: west,
# // //   framesPerSecond: 3,
# // //   min: 0,
# // //   max: 1,
# // //   palette: ['blue', 'red']
# // // };

# // // //Args for filmstrip
# // // var filmArgs = {
# // //   dimensions: 128,
# // //   region: conus,
# // //   min: 0,
# // //   max: 1,
# // //   palette: ['blue', 'red']
# // // };

# // // print(warmViz.getVideoThumbURL(videoArgs));
# // // print(warmViz.getFilmstripThumbURL(filmArgs));





# OLD CODE FOR TESTING & COMPARISON - still in GEE Javascript

# // // //////////////////////////////////////////////////////////////////////////////////////
# // // ///////////// PULL ALL VALUES FOR A FEW TEST POINTS TO CALCULATE ANOMALY BY HAND /////
# // // /////////////& COMPARE TO OUTPUT CALCULATION FROM R //////////////////////////////////
# // // //////////////////////////////////////////////////////////////////////////////////////

# // //Create feature collection from list of points
# // var pts = ee.FeatureCollection([
# //   ee.Feature(ee.Geometry.Point([-105.24249040058604,40.00981039217258]), {plot_id: "BoulderCO"}), //Boulder, CO, SEEC
# //   ee.Feature(ee.Geometry.Point([-110.786763, 43.432451]), {plot_id: "JacksonWY"}), //Jackson WY, Rafter J
# //   ee.Feature(ee.Geometry.Point([-122.486757, 48.733972]), {plot_id: "BellinghamWA"}), //Bellingham, WA, WWU
# //   ee.Feature(ee.Geometry.Point([-122.170020, 37.428193]), {plot_id: "StanfordCA"}), //Stanford, CA
# // ]);

# // //Function to buffer points
# // function bufferPoints(radius, bounds) {
# //   return function(pt) {
# //     pt = ee.Feature(pt);
# //     return bounds ? pt.buffer(radius).bounds() : pt.buffer(radius);
# //   };
# // }
# // //Buffer points by 15m radius to create visible areas
# // var buffPts = pts.map(bufferPoints(15, true));
# // //print("Points:", buffPts);

# // //Display data & points to ensure correct. Check a few values by hand
# // var testyr = 7;
# // //print('Anoms to take from', allanomsraw.get(0));
# // //Map.addLayer(ee.Image(ee.List(allanomsraw.get(5)).get(testyr - 1)), {min:0,max:500, palette:['blue', 'red']}, 'raw', false);
# // //Map.addLayer(ee.Image(ee.List(standardizeAll.get(5)).get(testyr - 1)), {min:0,max:500, palette:['blue', 'red']}, 'z-score', false);
# // //Map.addLayer(buffPts, {}, 'pts', true);

# // //print('Year is ', ee.Image(ee.List(allanomsraw.get(5)).get(testyr - 1)).get('year'));
# // //print('Band is ', ee.Image(ee.List(allanomsraw.get(5)).get(testyr - 1)).bandNames());


# // //Empty collection to fill
# // var ft = ee.FeatureCollection(ee.List([]));

# // //Function to fill feature collection with data from image collection
# // var fill = function(img, ini) {
# //   // type cast
# //   var inift = ee.FeatureCollection(ini);
# //   // gets the values for the points in the current img
# //   var ft2 = img.reduceRegions(pts, ee.Reducer.first(),30);
# //   // gets the year and variable of the img
# //   var yr = img.get('year');
# //   var v = (img.bandNames()).get(0);
# //   // writes the date in each feature
# //   var ft3 = ft2.map(function(f){return f.set("year", yr)});
# //   ft3 = ft3.map(function(f){return f.set("variable", v)});
# //   // merges the FeatureCollections
# //   return inift.merge(ft3);
# // };

# // //Function to use on list
# // var exportData = function(list) {
# //   var ic = ee.ImageCollection.fromImages(list);
# //   var thisft = ee.FeatureCollection(ic.iterate(fill,ft));
# //   return(thisft);
# // };

# // //Export all raw data for pts
# // var allRawExport = exportData(allanomsraw.get(0)).
# //   merge(exportData(allanomsraw.get(1))).
# //   merge(exportData(allanomsraw.get(2))).
# //   merge(exportData(allanomsraw.get(3))).
# //   merge(exportData(allanomsraw.get(4))).
# //   merge(exportData(allanomsraw.get(5)));
# // print('allRawExport', allRawExport);

# // //Export all z data for pts
# // var allZExport = exportData(standardizeAll.get(0)).
# //   merge(exportData(standardizeAll.get(1))).
# //   merge(exportData(standardizeAll.get(2))).
# //   merge(exportData(standardizeAll.get(3))).
# //   merge(exportData(standardizeAll.get(4))).
# //   merge(exportData(standardizeAll.get(5)));
# // print('allZExport', allZExport);

# // // //Export results
# // // Export.table.toDrive({
# // //   collection: allRawExport,
# // //   folder: foldername,
# // //   description: 'GEE_locations_calculated_raw_anomalies',
# // //   fileFormat: 'CSV'
# // // });
# // // Export.table.toDrive({
# // //   collection: allZExport,
# // //   folder: foldername,
# // //   description: 'GEE_locations_calculated_z_anomalies',
# // //   fileFormat: 'CSV'
# // // });